# KubiosCloud Demo 2
9.2.2023, Sakari Lukkarinen<br>
Health, Information and Communication Technology<br>
[Metropolia University of Applied Sciences](https://www.metropolia.fi/en)

## Introduction

The aim of this Notebook is to demonstrate how to ge data from KubiosCloud and show the measurement details. 

The code is based on the Kubioscloud example for authorization and Kubios Cloud API reference (see the References).

### How to use this Notebook

- **Before use, create a subfolder `json` where all collected data will be stored.**
- For USERNAME and PASSWORD use the Kubios HRV application username and password.
- You can use also the same username and password to login into the Kubios Cloud API.
- In addition you need CLIENT_ID to read the data from Kubioscloud.
- Run the code step-by-step.
- Some of the following code snippets needs info/data from the previous steps, so check carefully how the information is extracted from the JSON data-structures.
- At the end you should have all your analysis and raw data written into JSON and CSV files.

### References

- [Kubioscloud example for authorization](https://bitbucket.org/kubios/workspace/snippets/4X95xd/kubioscloud-example-for-authorization-code)
- [Kubios Cloud API reference](https://analysis.kubioscloud.com/v1/portal/documentation/apis.html#kubioscloud-api-reference)

## 1. Setup

If some of the libraries are missing, please install them:
- Write into to the Notebook cell: `!pip install ....`
- Open Anaconda command prompt and give command: `pip install ....`
- Open Anaconda Navigator > Environments and install missing packages

In [ ]:
#!/usr/bin/env python3

import base64
import logging
import re
import uuid
from pprint import pprint
import requests
import matplotlib.pyplot as plt
import numpy as np
import urllib
import json

In [ ]:
# Use your Kubios HRV App username and password
USERNAME = "..."
PASSWORD = "..."

# Client ID is given in a separate document found from OMA workspace
CLIENT_ID = "..."


LOGIN_URL = "https://kubioscloud.auth.eu-west-1.amazoncognito.com/login"
TOKEN_URL = "https://kubioscloud.auth.eu-west-1.amazoncognito.com/oauth2/token"
REDIRECT_URI = "https://analysis.kubioscloud.com/v1/portal/login"

USER_AGENT = "Hyte 2022"  # FIXME: Use unique name for your application

## 2. Opening a session and get tokens

In [ ]:
# Logging info
logging.basicConfig(format="%(asctime)-15s [%(levelname)s]: %(message)s")

log = logging.getLogger(__name__)
log.setLevel(logging.INFO)

csrf = str(uuid.uuid4())

## Login data structure
login_data = {
    "client_id": CLIENT_ID,
    "redirect_uri": REDIRECT_URI,
    "username": USERNAME,
    "password": PASSWORD,
    "response_type": "code",
    "access_type": "offline",
    "_csrf": csrf,
}

## Start a session
session = requests.session()

## Open a session
log.info("Authenticating to '%r' with client_id: %r", LOGIN_URL, CLIENT_ID)
login_response = session.post(
    LOGIN_URL,
    data=login_data,
    allow_redirects=False,
    headers={"Cookie": f"XSRF-TOKEN={csrf}", "User-Agent": USER_AGENT},
)

## Error handling
assert (
    login_response.status_code == 302
), f"Status: {login_response.status_code}, Authentication failed."

## Get the code
code = login_response.headers["Location"].split("=")[1]
log.info("Got code: %r", code)

## Exchange tokens
log.info("Exchanging code to tokens")
exch_data = {
    "client_id": CLIENT_ID,
    "code": code,
    "redirect_uri": REDIRECT_URI,
    "grant_type": "authorization_code",
}

exch_response = session.post(
    TOKEN_URL, data=exch_data
)
log.info("Status code %r", exch_response.status_code)
tokens = exch_response.json()

# What are the possible keys?
log.info(tokens.keys())

# When do these tokens expire?
log.info(f"Tokens expires in: {tokens['expires_in']} seconds.")

## Show the id_token
log.info("id_token: %r", tokens['id_token'])

## 3. Get user info and results

For more details, see:

- [KubiosCloud API reference](https://analysis.kubioscloud.com/v2/portal/documentation/apis.html#kubioscloud-api-reference)
- [Get user information](https://analysis.kubioscloud.com/v2/portal/documentation/apis.html#get-user-information)
- [Get results](https://analysis.kubioscloud.com/v2/portal/documentation/apis.html#get-results)
    - For daily readiness results, see the example at the end "Get results"
    
 NOTE: **You need to create a subfolder `json` where all collected data will be stored.**

In [ ]:
import datetime

d = datetime.datetime(2020,1,1,0,0)
print(d)
print(d.isoformat())

In [ ]:
## Create headers
HEADERS = {"Authorization": tokens["id_token"], "User-Agent": USER_AGENT}

## Create URLS
BASE_URL = "https://analysis.kubioscloud.com"
GET_USER_INFO = BASE_URL + "/v2/user/self"
GET_RESULT = BASE_URL + "/v2/result/self" + "?from=2020-01-01T00%3A00%3A00%2B00%3A00"
GET_DAILY_READINESS = BASE_URL + "/v2/result/self?types=readiness&daily=yes"


## Return personal information for the currently authenticated user ##
log.info("Get user info")
response = session.get(GET_USER_INFO, headers = HEADERS)
user_info = response.json()

with open("./json/user_info.json", "w") as outfile:
    json.dump(user_info, outfile)
log.info("Wrote user info to JSON file.")


## List all results for the current user from the last 30 days (all defaults) ##
log.info("Get results")
response = session.get(GET_RESULT, headers = HEADERS)
all_results = response.json()

# You need to create a subfolder json in order this works
with open("./json/all_results.json", "w") as outfile:
    json.dump(all_results, outfile)
log.info("Wrote results to JSON file.")


## List only “daily” readiness results ##
log.info("Get daily readiness results")
response = session.get(GET_DAILY_READINESS, headers = HEADERS)
daily_readiness = response.json()
   
with open("./json/daily_readiness.json", "w") as outfile:
    json.dump(daily_readiness, outfile)
log.info("Wrote daily readiness results to JSON file.")

## 3. All results

Read all measurement results from Kubios Cloud service and write them to JSON and CSV files into subfolder `json`.

In [ ]:
readiness = []
created = []
stress_index = []

# print('  n  Ready Stress   Timestamp')

for n, r in enumerate(all_results['results']):
    r_data = r['result']['readiness']
    c_stamp = r['create_timestamp']
    stress_i = r['result']['stress_index']
    readiness.append(r_data)
    created.append(c_stamp)
    stress_index.append(stress_i)
    filename = c_stamp[:19].replace(':','.')
    # print(f'{n:3d}  {r_data:5.0f}  {stress_i:5.1f}   {filename}')
    
    with open(f'./json/r_{n:0=3d}.json', "w") as outfile:
        json.dump(r, outfile)
        
    MEASURE_ID = r['measure_id']
    GET_RESULT_INFORMATION = BASE_URL + "/v2/measure/self/session/" + MEASURE_ID
    response = session.get(GET_RESULT_INFORMATION, headers = HEADERS)
    details = response.json()
    
    with open(f'./json/r_{n:0=3d}_details.json', "w") as outfile:
        json.dump(details, outfile)

    ## Read the DATA_URL and 'measured_timestamp'
    DATA_URL = details['measure']['channels'][0]['data_url']
    measured = details['measure']['measured_timestamp']

    ## Read the RR/PPI data from DATA_URL
    data = urllib.request.urlopen(DATA_URL)
    byte = data.read(2)
    rr = []

    ## Convert the binary data to numpy array
    while byte:
        rr.append(int.from_bytes(byte, byteorder = "little"))
        byte = data.read(2)
    rr = np.array(rr)
    
    np.savetxt(f'./json/r_{n:0=3d}_data.csv', rr, fmt="%d", delimiter=",")
    
    log.info(f'Wrote results {n:0=3d} JSON and CSV files.')

Next continue processing the data with *Reading JSON data.ipynb*.